# MultiIndexing (a.k.a Hierarchical Indexing)

A MultiIndex (a.k.a. Hierarchical Index) allows us to have multiple levels of indexing on a Series or DataFrame. Instead of having just one row or column label, you can have multiple labels per axis - making it possible to store and work with high-dimensional data in a 2D structure.

## Create a MultiIndex Series 

In [1]:
import pandas as pd 
import numpy as np

In [ ]:
index = [
    # a list of tuples 
    ('USA', 'New York'),
    ('USA', 'Los Angeles'),
    ('Canada', 'Toronto'),
    ('Canada', 'Vancouver')
]

sales = pd.Series([100, 200, 150, 170], index=index)
sales

(USA, New York)        100
(USA, Los Angeles)     200
(Canada, Toronto)      150
(Canada, Vancouver)    170
dtype: int64

This is not the preferred way - instead we should do a MultiIndexing !

In [4]:
index = pd.MultiIndex.from_tuples(index, names=['Country', 'City'])
sales = sales.reindex(index)
sales

Country  City       
USA      New York       100
         Los Angeles    200
Canada   Toronto        150
         Vancouver      170
dtype: int64

## Create a MultIndex DataFrame

In [9]:
arrays = [
    # a list of lists 
    ['Electronics', 'Electronics', 'Clothing', 'Clothing'],
    ['Q1', 'Q2', 'Q1', 'Q2']
]

columns = pd.MultiIndex.from_arrays(arrays, names=["Category", "Quarter",])

data = [
    [2000, 2500, 1500, 1800],  # Store A
    [2200, 2600, 1400, 1600]   # Store B
]

df_1= pd.DataFrame(data, columns=columns, index=['Store A', 'Store B'])
df_1

Category Electronics       Clothing      
Quarter           Q1    Q2       Q1    Q2
Store A         2000  2500     1500  1800
Store B         2200  2600     1400  1600

In [8]:
arrays = [
    ['Asia', 'Asia', 'Europe', 'Europe'],
    ['India', 'China', 'France', 'Germany']
]

index = pd.MultiIndex.from_arrays(arrays, names=('Continent', 'Country'))
df_2 = pd.DataFrame({'Population': [1.4, 1.3, 0.07, 0.08]}, index=index)

df_2

Population
Continent Country            
Asia      India          1.40
          China          1.30
Europe    France         0.07
          Germany        0.08

## Indexing and Slicing in MultiIndex

In [ ]:
# direct access using .loc[]
df_2.loc[('Asia', 'India')] # You must provide a tuple for hierarchical levels.

# same as df_2.loc[('Asia', 'India'),:]

Population    1.4
Name: (Asia, India), dtype: float64

In [ ]:
# partial Indexing 
df_2.loc['Asia'] # same as f_2.lpc['Asia',:]

,Population
Country,
China,1.3
India,1.4


In [12]:
# Slicing 
df_2.loc[('Asia', slice(None))]

,Population
Country,
India,1.4
China,1.3


| Method                                            | Includes `('Asia', 'India')`? | Why                                 |
| ------------------------------------------------- | ----------------------------- | ----------------------------------- |
| `df.loc[idx['Asia':'Europe', 'China':'Germany']]` | ❌                             | Slices each level independently     |
| `df.loc[('Asia', 'China'):('Europe', 'Germany')]` | ✅                             | Lexicographical slice of full tuple |


In [34]:
df_2_sorted = df_2.sort_index()
df_2_sorted.loc[('Asia','China') : ('Europe','Germany'),:]

Population
Continent Country            
Asia      China          1.30
          India          1.40
Europe    France         0.07
          Germany        0.08

In [35]:
# select a range 
idx = pd.IndexSlice
df_2_sorted.loc[idx['Asia':'Europe', 'China':'Germany'], :]

Population
Continent Country            
Asia      China          1.30
Europe    France         0.07
          Germany        0.08

In [30]:
df_sorted = df_2.sort_index()
df_sorted.loc[('Asia', 'China'):('Europe', 'France')]

Population
Continent Country            
Asia      China          1.30
          India          1.40
Europe    France         0.07

### 🔍 Question:

> If `'India'` comes after `'China'`, and I’m slicing from `'China'` to `'Germany'`, **shouldn't `'India'` be included**?

---

### 🧠 Lexicographic Ordering in Action

Imagine the tuples arranged as a list:

```python
[
  ('Asia', 'China'),   ✅ START
  ('Asia', 'India'),   ✅ falls within range!
  ('Europe', 'France'),
  ('Europe', 'Germany') ✅ END
]
```

Looks like it should include `('Asia', 'India')`, right?

> **Yes — if** you wrote the slicing as:
> `df.loc[idx[('Asia', 'China'):('Europe', 'Germany')]]`
> (a tuple range instead of level-wise slicing)

---

#### ❗But you did this instead:

```python
df.loc[idx['Asia':'Europe', 'China':'Germany']]
```

And this is **NOT** a slice from tuple `('Asia', 'China')` to `('Europe', 'Germany')`.

Instead, it's interpreted as:

* For **level 0**: `'Asia'` to `'Europe'`
* For **level 1**: `'China'` to `'Germany'`

But the slicing happens **within each level**, not across the full tuple directly.

So here’s what Pandas does:

| Level 0 (Continent) | Level 1 (Country) | Keep? | Why                                      |
| ------------------- | ----------------- | ----- | ---------------------------------------- |
| Asia                | China             | ✅     | Within slice                             |
| Asia                | India             | ❌     | `'India'` is outside `'China':'Germany'` |
| Europe              | France            | ✅     | Within slice                             |
| Europe              | Germany           | ✅     | Within slice                             |

⚠️ The condition is: **Continent must be in `'Asia':'Europe'` AND Country in `'China':'Germany'`**, **independently**.

`'India'` is outside `'China':'Germany'` alphabetically, so it’s excluded.

---

### ✅ If You Want `'Asia', 'India'` Included

You should slice the **tuples**, not levels:

```python
df_sorted = df.sort_index()
df_sorted.loc[('Asia', 'China'):('Europe', 'Germany')]
```

Now you’ll get:

```
                     Population
Continent Country             
Asia      China            1.30
          India            1.40
Europe    France           0.07
          Germany          0.08
```

✅ **Boom! `'India'` is included.**

---

### 🧾 Summary

| Method                                            | Includes `('Asia', 'India')`? | Why                                 |
| ------------------------------------------------- | ----------------------------- | ----------------------------------- |
| `df.loc[idx['Asia':'Europe', 'China':'Germany']]` | ❌                             | Slices each level independently     |
| `df.loc[('Asia', 'China'):('Europe', 'Germany')]` | ✅                             | Lexicographical slice of full tuple |

---

### 🧠 Takeaway

> If you want to slice using **tuples**, use `df.loc[start_tuple:end_tuple]` after sorting the index.

> If you slice **level by level** using `IndexSlice`, remember: each level is sliced **independently**, and this may cause unexpected exclusions.

---

### Column Indexing with MultiIndex Columns

In [36]:
columns = pd.MultiIndex.from_tuples([
    ('Sales', 'Q1'), ('Sales', 'Q2'),
    ('Profit', 'Q1'), ('Profit', 'Q2')
])

df = pd.DataFrame(
    [[100, 150, 10, 15], [200, 250, 20, 25]],
    columns=columns,
    index=['Product A', 'Product B']
)

df

Sales      Profit    
             Q1   Q2     Q1  Q2
Product A   100  150     10  15
Product B   200  250     20  25

In [37]:
# 🎯 Accessing a full level:
df['Sales']

,Q1,Q2
Product A,100,150
Product B,200,250


In [ ]:
df.loc[:,'Sales'] # same as above 

,Q1,Q2
Product A,100,150
Product B,200,250


In [39]:
# 🔍 Accessing a specific cell:
df.loc['Product A', ('Sales', 'Q1')]

np.int64(100)

In [ ]:
# slicing 
df.loc[:, ('Sales', slice(None))]

Sales     
             Q1   Q2
Product A   100  150
Product B   200  250

In [56]:
# boolean masking 
df_2

Population
Continent Country            
Asia      China          1.30
          India          1.40
Europe    France         0.07
          Germany        0.08

In [58]:
df_2[df_2['Population']> 1]

Population
Continent Country            
Asia      China           1.3
          India           1.4

## Using .xs() (Cross-Section)

In [59]:
df_2.xs(key='Asia', axis=0)

,Population
Country,
China,1.3
India,1.4


In [60]:
df_2.xs(key='Asia', level='Continent')

,Population
Country,
China,1.3
India,1.4


In [ ]:
df.xs('Q1', axis=1, level=1)

,Sales,Profit
Product A,100,10
Product B,200,20



## 🔄 Reshaping with MultiIndex — Deep Dive

MultiIndexes allow for powerful reshaping with operations like:

| Operation                     | Description                                     |
| ----------------------------- | ----------------------------------------------- |
| `.stack()`                    | Move columns into the row index                 |
| `.unstack()`                  | Move index levels into columns                  |
| `.pivot()` / `.pivot_table()` | Reshape data by pivoting one level into columns |
| `.melt()`                     | Unpivot the DataFrame                           |
| `.swaplevel()`                | Swap two index levels                           |
| `.sort_index()`               | Sort by specific index levels                   |
| `.reorder_levels()`           | Reorder levels of index/columns                 |


In [62]:
arrays = [
    ['North', 'North', 'South', 'South'],
    ['Jan', 'Feb', 'Jan', 'Feb']
]
index = pd.MultiIndex.from_arrays(arrays, names=('Region', 'Month'))

df = pd.DataFrame({
    'Sales': [250, 270, 300, 310],
    'Profit': [25, 30, 35, 40]
}, index=index)

print(df)



              Sales  Profit
Region Month               
North  Jan      250      25
       Feb      270      30
South  Jan      300      35
       Feb      310      40


In [64]:
# unstack() - turns row index into columns 
df_unstack = df.unstack(level='Month')
df_unstack

Sales      Profit    
Month    Feb  Jan    Feb Jan
Region                      
North    270  250     30  25
South    310  300     40  35

In [67]:
# stack() - Turn Columns into Row Index
df_unstack.stack(future_stack=True)

Sales  Profit
Region Month               
North  Feb      270      30
       Jan      250      25
South  Feb      310      40
       Jan      300      35